# 撰寫爬蟲，收集網路上各大報的「熱門新聞」，並將它們存入 MongoDB 資料庫中









##### 1. 載入套件

In [ ]:
!pip install beautifulsoup4
import requests
from bs4 import BeautifulSoup
import json

##### 2. 爬取網站

*   模型: OpenAI ChatGPT
*   提詞: 幫我寫一個可以寫成在colab可以執行的python程式碼，並且做到以下事項:



> 1.   抓取此網址https://www.ettoday.net/news/hot-news.htm & https://udn.com/rank/pv/2
>2.   抓出每則熱門新聞的「標題」、「超連結」、「發表時間」，並標註「資料來源」以json格式印出

###### 2.1 ETtoday

In [ ]:
ETtoday_url = "https://www.ettoday.net/news/hot-news.htm"
ETtoday_text=requests.get(ETtoday_url).text
print(ETtoday_text)

In [ ]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

# 發送請求
response = requests.get(ETtoday_url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    enews_list = []

    # 抓取新聞標題、連結、發表時間
    for article in soup.find_all('div','piece clearfix'):
        date = article.find("span", class_="date").text.strip() if article.find("span") else None
        title_tag = article.find("h3").find("a") if article.find("h3") else None
        title = title_tag.text.strip() if title_tag else ""
        link = title_tag["href"] if title_tag else ""
        summary = article.find("p", class_="summary").text.strip() if article.find("p", class_="summary") else ""

        enews_list.append({
            "title": title,
            "link": link,
            "summary": summary,
            "time": date,
            "source": "ETtoday"
        })
    # 以 JSON 格式輸出
    print(json.dumps(enews_list, ensure_ascii=False, indent=4))
else:
    print("Failed to retrieve the webpage.")


###### 2.2 udn

In [ ]:
udn_url = "https://udn.com/rank/pv/2"
udn_text=requests.get(udn_url).text
print(udn_text)

In [ ]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

# 發送請求
response = requests.get(udn_url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    unews_list = []

    # 抓取新聞標題、連結、發表時間
    for article in soup.find_all('div', class_='story-list__news'):
        date = article.find('time', class_='story-list__time')

        # 檢查 h2 標籤是否存在
        h2_tag = article.find("h2")
        if h2_tag:
            title_tag = h2_tag.find("a")
            if title_tag:
                title = title_tag.text.strip()
                link = title_tag.get('href')
                date = date.text.strip() if date else None

                # 加入新聞列表
                unews_list.append({
                    "title": title,
                    "link": link,
                    "time": date,
                    "source": "UDN"
                })

    # 以 JSON 格式輸出
    print(json.dumps(unews_list, ensure_ascii=False, indent=4))
else:
    print("Failed to retrieve the webpage.")

[
    {
        "title": "他求助「嘴破有什麼特效藥？」 網推1水果：真的超有效",
        "link": "https://udn.com/news/story/7266/8603187",
        "time": "2025-03-12 17:24",
        "source": "UDN"
    },
    {
        "title": "立委羅廷瑋罵人？台大兒科總醫師「具名還原」事發過程",
        "link": "https://udn.com/news/story/6656/8603004",
        "time": "2025-03-12 14:39",
        "source": "UDN"
    },
    {
        "title": "「眾量級」Andy控頻道遭前女友一家整碗端走 家寧父昨晚突報警",
        "link": "https://udn.com/news/story/7320/8604980",
        "time": "2025-03-13 11:33",
        "source": "UDN"
    },
    {
        "title": "金秀賢有戀童癖？遭爆與金賽綸交往6年 醫曝這類人有3特徵",
        "link": "https://udn.com/news/story/7266/8603740",
        "time": "2025-03-12 19:42",
        "source": "UDN"
    },
    {
        "title": "新接任英特爾CEO的陳立武是誰？曾獲評科技界人脈最廣高階主管",
        "link": "https://udn.com/news/story/6811/8604480",
        "time": "2025-03-13 08:09",
        "source": "UDN"
    },
    {
        "title": "明起4天雷雨狂炸！周日低溫急凍僅10度 下周挑戰寒流",
        "link": "https://udn.com/news/st

##### 3. 安裝 MongoDB & PyMongo

In [ ]:
!wget -qO - https://fastdl.mongodb.org/linux/mongodb-linux-x86_64-ubuntu2204-7.0.7.tgz | tar -xzv
!mv mongodb-linux-x86_64-ubuntu2204-7.0.7/bin/* /usr/bin/

!wget https://www.mongodb.org/static/pgp/server-7.0.asc
!sudo apt-key add server-7.0.asc

# 建造 MongoDB 資料庫存放處（路徑、名稱可自訂）
!mkdir -p /content/data/db
# 啟動 MongoDB 伺服器（Mongo Daemon）於本地端（localhost）
!mongod --fork --logpath /var/log/mongodb.log --dbpath /content/data/db --bind_ip localhost

from google.colab import drive
drive.mount('/content/gdrive')

# 啟動 MongoDB 伺服器（Mongo Daemon）於本地端（localhost）（使用掛載的外部硬碟會耗時約 1 分鐘，請耐心等待）
!mongod --fork --logpath /var/log/mongodb.log --dbpath /content/gdrive/MyDrive/Datasets/MongoDB --bind_ip localhost

!mongod --sysinfo

!pip install pymongo

mongodb-linux-x86_64-ubuntu2204-7.0.7/LICENSE-Community.txt
mongodb-linux-x86_64-ubuntu2204-7.0.7/MPL-2
mongodb-linux-x86_64-ubuntu2204-7.0.7/README
mongodb-linux-x86_64-ubuntu2204-7.0.7/THIRD-PARTY-NOTICES
mongodb-linux-x86_64-ubuntu2204-7.0.7/bin/install_compass
mongodb-linux-x86_64-ubuntu2204-7.0.7/bin/mongod
mongodb-linux-x86_64-ubuntu2204-7.0.7/bin/mongos
--2025-03-13 05:44:25--  https://www.mongodb.org/static/pgp/server-7.0.asc
Resolving www.mongodb.org (www.mongodb.org)... 54.175.147.155, 52.206.222.245, 52.21.89.200
Connecting to www.mongodb.org (www.mongodb.org)|54.175.147.155|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://pgp.mongodb.com/server-7.0.asc [following]
--2025-03-13 05:44:26--  https://pgp.mongodb.com/server-7.0.asc
Resolving pgp.mongodb.com (pgp.mongodb.com)... 18.238.176.72, 18.238.176.124, 18.238.176.118, ...
Connecting to pgp.mongodb.com (pgp.mongodb.com)|18.238.176.72|:443... connected.
HTTP request sent, awa

##### 4. 連上 MongoDB 資料庫

In [ ]:
# 匯入 pymongo 的 MongoClient 物件
from pymongo import MongoClient
import pprint  # 用來美化輸出 server info

try:
    # 1. 連上 MongoDB 伺服器（請確認連線字串正確）
    client = MongoClient("mongodb://localhost:27017/")

    # 2. 顯示 MongoDB 伺服器資訊，確認連線成功
    print("MongoDB 伺服器資訊:")
    pprint.pprint(client.server_info())  # 若連線失敗，此行會拋出例外

    # 3. 取得並印出所有現存的資料庫名稱
    print("\n現有的資料庫:")
    db_list = client.list_database_names()
    for db in db_list:
        print("- " + db)

except Exception as e:
    print("連線錯誤:", e)


##### 5. 建立資料庫
參考老師講義的程式碼




In [ ]:
from pymongo import MongoClient
import pprint  # 用於美化輸出

try:

    # 建立一個名為 'News' 的資料庫
    db = client['News']

    # 在 'News' 資料庫中建立 'ETtoday' 資料表
    ETtoday= db['ETtoday']
    udn= db['udn']

    ETtoday .delete_many({})
    udn.delete_many({})

    # 插入新聞資料到 'ETtoday' 資料表
    ETtoday.insert_many(enews_list)
    udn.insert_many(unews_list)

    # 驗證資料庫及資料表建立成功
    # 印出所有資料庫名稱
    print("所有的資料庫：")
    pprint.pprint(client.list_database_names())

    # 印出 'ETtoday' 資料庫中的所有資料表名稱
    print("\n'News' 資料庫中的所有資料表：")
    pprint.pprint(db.list_collection_names())

    # 印出 'ETtoday' 資料表內所有資料列
    print("\n'ETtoday' 資料表中的資料：")
    for doc in ETtoday.find():
        pprint.pprint(doc)

    # 印出 'udn' 資料表內所有資料列
    print("\n'udn' 資料表中的資料：")
    for doc in udn.find():
        pprint.pprint(doc)

except Exception as e:
    print("發生錯誤：", e)


所有的資料庫：
['News', 'admin', 'config', 'local']

'News' 資料庫中的所有資料表：
['udn', 'ETtoday']

'ETtoday' 資料表中的資料：
{'_id': ObjectId('67d26f25d85d586c9f5fe351'),
 'link': 'https://www.ettoday.net/news/20250313/2924545.htm',
 'source': 'ETtoday',
 'summary': 'YouTuber眾量級Andy、家寧在2024年10月官宣分手，男方11日在個人頻道開撕，怒控家寧一家人的一連串作為讓他失去頻道的經營權，同時他發出「珍惜生命聲明」。而家寧的父親12日深夜報警，表示有..',
 'time': '2小時前',
 'title': '4男工廠外打手遊等友下班…家寧爸報警喊騷擾！房仲嗆：作賊心虛'}
{'_id': ObjectId('67d26f25d85d586c9f5fe352'),
 'link': 'https://star.ettoday.net/news/2924387',
 'source': 'ETtoday',
 'summary': '網紅眾量級Andy '
            '11日拋出震撼彈，指控前女友家寧一家的8大罪狀，並坦承已失去控制辛苦經營10年頻道的權利。事件經過發酵，家寧仍未發文回應，卻被網友發現她似乎刪除或隱藏了一些貼文。',
 'time': '4小時前',
 'title': '家寧「神隱1天出手了」！被抓包22貼文消失…網全截圖：滅證中嗎'}
{'_id': ObjectId('67d26f25d85d586c9f5fe353'),
 'link': 'https://www.ettoday.net/news/20250312/2924102.htm',
 'source': 'ETtoday',
 'summary': '眾量級Andy指控，辛苦經營頻道10年，期間卻只領3萬至5萬5000元的月薪，分手後更是一腳被踢開，「一個百萬YTR年平均賺到8位數算合理吧？」網友們同時也翻出家寧妹在IG曬出千萬豪車、勞力士照片，甚..',
 'time': '20小時前',
 'title': '再挖家寧妹黑歷史

###### 6. 讀取資料庫
*   模型: OpenAI ChatGPT
*   提詞: 承接剛剛的程式碼，幫我將 'news' 資料庫中的「新聞標題」，「日期」，「網址」和「來源」的資訊全部列出來。


In [ ]:
for news in ETtoday.find({}, {"_id": 0, "title": 1, "time": 1, "link": 1, "source": 1}):
    print("---------------------------")
    print(f"標題: {news['title']}")
    print(f"日期: {news['time']}")
    print(f"網址: {news['link']}")
    print(f"來源: {news['source']}")

for news in udn.find({}, {"_id": 0, "title": 1, "time": 1, "link": 1, "source": 1}):
    print("---------------------------")
    print(f"標題: {news['title']}")
    print(f"日期: {news['time']}")
    print(f"網址: {news['link']}")
    print(f"來源: {news['source']}")

---------------------------
標題: 4男工廠外打手遊等友下班…家寧爸報警喊騷擾！房仲嗆：作賊心虛
日期: 2小時前
網址: https://www.ettoday.net/news/20250313/2924545.htm
來源: ETtoday
---------------------------
標題: 家寧「神隱1天出手了」！被抓包22貼文消失…網全截圖：滅證中嗎
日期: 4小時前
網址: https://star.ettoday.net/news/2924387
來源: ETtoday
---------------------------
標題: 再挖家寧妹黑歷史「1句反咬Andy」分手內幕！網怒：真敢講
日期: 20小時前
網址: https://www.ettoday.net/news/20250312/2924102.htm
來源: ETtoday
---------------------------
標題: 徐睿知爆當金賽綸小三「崩潰首發聲」！　痛訴金秀賢感情狀態：停止吧
日期: 5小時前
網址: https://star.ettoday.net/news/2924381
來源: ETtoday
---------------------------
標題: Andy選311開戰家寧「超暖原因瘋傳」！　網刷讚：對她最後的溫柔
日期: 3小時前
網址: https://star.ettoday.net/news/2924436
來源: ETtoday
---------------------------
標題: 家寧媽偷補法律知識被抓包？眾量級頻道新增「創業股權分配」影片
日期: 15小時前
網址: https://www.ettoday.net/news/20250312/2924310.htm
來源: ETtoday
---------------------------
標題: Andy控家寧媽私註冊商標！律師曝「剩20天搶救」　智慧局給答案了
日期: 12小時前
網址: https://www.ettoday.net/news/20250313/2924332.htm
來源: ETtoday
---------------------------
標題: 快訊／金秀賢3度強硬發聲！　「握有明確證據」：下週統一